<a href="https://colab.research.google.com/github/tomthomas/Coursera_Capstone/blob/main/Toronto_Cluster_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Clustering Neighborhoods in Toronto 



We will work on extracting data from wiki which contains postal code information for all the neighborhoods of Toronto.

After extraction, it will be converted into a Dataframe. 
After the Dataframe is created, we will add geospatial data to it. 

Once the Dataframe is sorted, we will use the Foursquare API and Folium to visualize the data into applicable clusters.

In [ ]:
#importing requried libraries

import pandas as pd
import requests
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans

print("Import complete!")

Import complete!


Import page data and convert into a dataframe

In [ ]:
page = requests.get('http://zims-en.kiwix.campusafrica.gos.orange.com/wikipedia_en_all_nopic/A/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(page,'html5lib')
tbl = str(soup.table)

pc0 = pd.read_html(tbl)
pc = pc0[0]
pc.head()

Data Cleaning - Create Data Frame Using directions given in assignment

In [ ]:
pc1 = pc[pc.Borough != 'Not assigned']
pc1 = pc1.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
pc1.reset_index(inplace=True)
pc1['Neighbourhood'] = np.where(pc1['Neighbourhood'] == 'Not assigned',pc1['Borough'], pc1['Neighbourhood'])

Import Geospatial DataSet

In [ ]:
geo = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge Tables to create required DataFrame

In [ ]:
geo.rename(columns = {'Postal Code':'Postcode'}, inplace = True)
pc_clean = pd.merge(pc1, geo, on = 'Postcode')
pc_clean.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Cluster Data using KMeans

In [ ]:
k=5
toronto_cluster = pc_clean[['Latitude','Longitude']]
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_cluster)
kmeans.labels_
pc_clean.insert(0, 'Cluster Labels', kmeans.labels_)

DataFrame with Clusters

In [ ]:
pc_clean.head()

,Cluster Labels,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,0,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,2,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Visualize DataFrame using Folium

In [ ]:
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(pc_clean['Latitude'], pc_clean['Longitude'], pc_clean['Neighbourhood'], pc_clean['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(toronto)
toronto